# Linjer Regresson filled nan values 

In [1]:
from taxipred.utils.constants import LINJER_FILLED_NA
import pandas as pd 
df = pd.read_csv(LINJER_FILLED_NA)

df.head(5)

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price,Day_of_Week_Weekend,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night
0,19.35,3.0,3.56,0.80,0.32,53.82,36.2624,0,0,1,0
1,36.87,1.0,2.70,1.21,0.15,37.27,52.9032,1,1,0,0
2,30.33,4.0,3.48,0.51,0.15,116.81,36.4698,0,1,0,0
3,NaN,3.0,2.93,0.63,0.32,22.64,15.6180,0,1,0,0
4,8.64,2.0,2.55,1.71,0.48,89.33,60.2028,1,0,0,0


In [3]:
X, y = df.drop("Trip_Price", axis="columns"), df["Trip_Price"]
X.head(2), y.head(2)


(   Trip_Distance_km  Passenger_Count  Base_Fare  Per_Km_Rate  Per_Minute_Rate  \
 0             19.35              3.0       3.56         0.80             0.32   
 1             36.87              1.0       2.70         1.21             0.15   
 
    Trip_Duration_Minutes  Day_of_Week_Weekend  Time_of_Day_Evening  \
 0                  53.82                    0                    0   
 1                  37.27                    1                    1   
 
    Time_of_Day_Morning  Time_of_Day_Night  
 0                    1                  0  
 1                    0                  0  ,
 0    36.2624
 1    52.9032
 Name: Trip_Price, dtype: float64)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

print(f"{X_train.shape = }")
print(f"{X_test.shape = }")
print(f"{y_train.shape = }")
print(f"{y_test.shape = }")

X_train.shape = (637, 10)
X_test.shape = (314, 10)
y_train.shape = (637,)
y_test.shape = (314,)


Gonna fill nan values here but only in x traib/test to prevet datalekage 

In [5]:
values_mean = X_train.mean()

X_train_nonan = X_train.fillna(values_mean)
X_test_nonan = X_test.fillna(values_mean)

In [6]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
scale.fit(X_train_nonan)
scaled_X_train = scale.transform(X_train_nonan)
scaled_X_test = scale.transform(X_test_nonan)

scaled_X_test.shape, scaled_X_train.shape

((314, 10), (637, 10))

In [7]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(scaled_X_train,y_train)
model.coef_, model.intercept_

(array([34.8726358 ,  0.47828672,  0.32938148, 10.65318043,  6.30872179,
         9.48172624, -0.73299109, -0.86839809, -0.22380331, -1.32762732]),
 np.float64(55.48228606344443))

In [8]:
test_sample_1 = scaled_X_test[0].reshape(1,-1)
test_sample_2 = y_test.values[0]
test_sample_1, test_sample_2

(array([[-0.37947624, -1.36109827, -1.29150872, -0.88613607,  0.00384842,
         -1.39351082, -0.64463599, -0.50636968, -0.62273555, -0.31949354]]),
 np.float64(24.0367))

In [9]:
pred_y = model.predict(scaled_X_test)
pred_y

array([ 20.01955956,  51.57625308,  40.06646631,  39.17326084,
        76.15318125,  17.38681967,  12.53136184,  63.86054248,
        50.47292789,  73.29131224, 192.6837191 ,  52.65040828,
        29.63591914,  54.18326546, -18.47584416,  71.92517923,
        58.38676149,  63.22729551,  37.56557844,  61.74781668,
        67.11765229,  57.59676104, 208.21175084,  41.92528483,
        87.45960114, 189.05316803,  37.01795398,  33.40957233,
        62.08211343,  36.39434559,  40.62923225,  64.03016266,
        81.63967453,  81.51840479,  93.88136838,  39.20933969,
        47.9739597 ,  58.67863914,  37.99886767,  89.58330206,
        47.7321263 ,  43.85962434,  48.93902262,  59.82530423,
        32.65880429,  78.42032981,  57.40445359,  57.3681265 ,
        51.22159682,  52.81004558,  68.50176619,  54.12562511,
        88.53324476,  18.13888078,  53.56900444,  79.52195485,
       112.41956453,  85.02259065,  33.99874311,  84.40371198,
        98.19665709,  14.21175407,  31.50752666,  13.64

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mean_ab_er = mean_absolute_error(y_test,pred_y)
mean_sq_er = mean_squared_error(y_test,pred_y)
root_me_sq_er = np.sqrt(mean_sq_er)

print(f"Mean absolut error {mean_ab_er}, Mean squarerd error {mean_sq_er}, Root Mean squared error {root_me_sq_er}")

Mean absolut error 9.510847547245534, Mean squarerd error 215.57370217114592, Root Mean squared error 14.682428347216476


In [11]:
base_line = np.mean(y_test)
base_with_mae = mean_absolute_error(y_test, [base_line]*len(y_test))
base_line, base_with_mae

(np.float64(59.6996599055302), 25.07280855350934)